# Apply Group By Pandas

In [4]:
import pandas as pd
import numpy as np

In [5]:
pd.__version__

'1.0.1'

In [6]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
pd.options.display.float_format = '{:,.3f}'.format

In [8]:
df.describe(include='all')

,total_bill,tip,sex,smoker,day,time,size
count,244.000,244.000,244,244,244,244,244.000
unique,nan,nan,2,2,4,2,nan
top,nan,nan,Male,No,Sat,Dinner,nan
freq,nan,nan,157,151,87,176,nan
mean,19.786,2.998,NaN,NaN,NaN,NaN,2.570
std,8.902,1.384,NaN,NaN,NaN,NaN,0.951
min,3.070,1.000,NaN,NaN,NaN,NaN,1.000
25%,13.348,2.000,NaN,NaN,NaN,NaN,2.000
50%,17.795,2.900,NaN,NaN,NaN,NaN,2.000
75%,24.127,3.562,NaN,NaN,NaN,NaN,3.000


In [9]:
df['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [10]:
df.groupby('sex')['total_bill', 'tip'].mean()

,total_bill,tip
sex,,
Male,20.744,3.090
Female,18.057,2.833


In [11]:
df['porcentaje_propina'] = df['tip'] / df['total_bill']
df.head()

,total_bill,tip,sex,smoker,day,time,size,porcentaje_propina
0,16.990,1.010,Female,No,Sun,Dinner,2,0.059
1,10.340,1.660,Male,No,Sun,Dinner,3,0.161
2,21.010,3.500,Male,No,Sun,Dinner,3,0.167
3,23.680,3.310,Male,No,Sun,Dinner,2,0.140
4,24.590,3.610,Female,No,Sun,Dinner,4,0.147


In [13]:
df.groupby('sex')['porcentaje_propina'].describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
Male,157.000,0.158,0.065,0.036,0.121,0.153,0.186,0.710
Female,87.000,0.166,0.054,0.056,0.140,0.156,0.194,0.417


In [14]:
def mean_eur2usd(x):
    return np.mean(x) * 1.12

In [15]:
df.groupby('sex')['total_bill'].apply(mean_eur2usd)

sex
Male     23.233
Female   20.224
Name: total_bill, dtype: float64

In [16]:
df.groupby('sex')['total_bill'].apply(lambda x : np.mean(x) * 1.12)

sex
Male     23.233
Female   20.224
Name: total_bill, dtype: float64

In [17]:
def f_filter(x):
    return mean_eur2usd(x['total_bill'].mean()) > 20

In [18]:
df.groupby(['sex', 'time']).filter(f_filter)

,total_bill,tip,sex,smoker,day,time,size,porcentaje_propina
0,16.990,1.010,Female,No,Sun,Dinner,2,0.059
1,10.340,1.660,Male,No,Sun,Dinner,3,0.161
2,21.010,3.500,Male,No,Sun,Dinner,3,0.167
3,23.680,3.310,Male,No,Sun,Dinner,2,0.140
4,24.590,3.610,Female,No,Sun,Dinner,4,0.147
...,...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3,0.204
240,27.180,2.000,Female,Yes,Sat,Dinner,2,0.074
241,22.670,2.000,Male,Yes,Sat,Dinner,2,0.088
242,17.820,1.750,Male,No,Sat,Dinner,2,0.098


In [4]:
df['ones'] = 1

In [7]:
df_g = df.groupby(['sex', 'smoker'])['ones'].sum()
df_g

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
Name: ones, dtype: int64

In [12]:
# Convertir una variable continua en rangos de clase
pd.cut(df['total_bill'], bins=3).value_counts()

(3.022, 18.983]     140
(18.983, 34.897]     88
(34.897, 50.81]      16
Name: total_bill, dtype: int64

In [15]:
pd.cut(df['total_bill'], bins=[3, 18, 35, 60]).value_counts()

(3, 18]     127
(18, 35]    101
(35, 60]     16
Name: total_bill, dtype: int64

## Pivot Table

In [14]:
df.groupby(['sex', 'time'])['total_bill'].mean()

sex     time  
Male    Lunch    18.048
        Dinner   21.461
Female  Lunch    16.339
        Dinner   19.213
Name: total_bill, dtype: float64

In [17]:
df_gp = df.groupby(['sex', 'time'])['total_bill'].mean().reset_index()
df_gp

,sex,time,total_bill
0,Male,Lunch,18.048
1,Male,Dinner,21.461
2,Female,Lunch,16.339
3,Female,Dinner,19.213


In [18]:
df_gp.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.048,21.461
Female,16.339,19.213


In [19]:
df.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.048,21.461
Female,16.339,19.213


In [21]:
df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc=np.median)

time,Lunch,Dinner
sex,,
Male,16.580,19.630
Female,13.420,17.190


In [24]:
df_pivot = df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc=[np.median, np.std])
df_pivot

median          std       
time    Lunch Dinner Lunch Dinner
sex                              
Male   16.580 19.630 7.953  9.461
Female 13.420 17.190 7.501  8.202

In [28]:
df_pivot.unstack().reset_index()
df_pivot

median          std       
time    Lunch Dinner Lunch Dinner
sex                              
Male   16.580 19.630 7.953  9.461
Female 13.420 17.190 7.501  8.202